In [2]:
def get_synonyms(word, postag):
    synsets = wordnet.synsets(word, pos=postag)
    synonyms = {lemma.name().replace('_', ' ') for syn in synsets for lemma in syn.lemmas()}
    return list(synonyms)


In [ ]:
import nltk
from nltk.corpus import wordnet
import random


ModuleNotFoundError: No module named 'pattern'

In [4]:
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /Users/lsc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/lsc/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
nlp = spacy.load("en_core_web_sm")


In [36]:
def substitute_words(text, p_level):
    """Perform static substitution on eligible words."""
    doc = nlp(text)
    perturbed_text = []
    
    for token in doc:
        if token.pos_ in {"ADJ", "VERB", "ADV"} and random.random() < p_level: 
            pos_map = {"ADJ": wordnet.ADJ, "VERB": wordnet.VERB, "ADV": wordnet.ADV}
            synonyms = get_synonyms(token.text, pos_map[token.pos_])
            if synonyms:
                # Choose a random synonym and preserve the token's original spacing
                perturbed_text.append(synonyms[random.randint(0, len(synonyms) - 1)] + token.whitespace_)
            else:
                perturbed_text.append(token.text + token.whitespace_)
        else:
            perturbed_text.append(token.text + token.whitespace_)
    
    return "".join(perturbed_text)


In [37]:
input_text = """If you're visiting this page, you're likely here because you're searching for a random sentence. Sometimes a random word just isn't enough, and that is where the random sentence generator comes into play. By inputting the desired number, you can make a list of as many random sentences as you want or need. Producing random sentences can be helpful in a number of different ways. For writers, a random sentence can help them get their creative juices flowing. Since the topic of the sentence is completely unknown, it forces the writer to be creative when the sentence appears. There are a number of different ways a writer can use the random sentence for creativity. The most common way to use the sentence is to begin a story. Another option is to include it somewhere in the story. A much more difficult challenge is to use it to end a story. In any of these cases, it forces the writer to think creatively since they have no idea what sentence will appear from the tool.

"""
perturbed_output = substitute_words(input_text, 0.6)
print("Original:", input_text)
print("Perturbed:", perturbed_output)


Original: If you're visiting this page, you're likely here because you're searching for a random sentence. Sometimes a random word just isn't enough, and that is where the random sentence generator comes into play. By inputting the desired number, you can make a list of as many random sentences as you want or need. Producing random sentences can be helpful in a number of different ways. For writers, a random sentence can help them get their creative juices flowing. Since the topic of the sentence is completely unknown, it forces the writer to be creative when the sentence appears. There are a number of different ways a writer can use the random sentence for creativity. The most common way to use the sentence is to begin a story. Another option is to include it somewhere in the story. A much more difficult challenge is to use it to end a story. In any of these cases, it forces the writer to think creatively since they have no idea what sentence will appear from the tool.


Perturbed: If

In [23]:
def reorder_sentences(text):
    doc = nlp(text)
    
    independent = []
    dependent = []
    for sent in doc.sents:
        if any(tok.dep_ in {"mark", "prep", "cc"} for tok in sent): 
            dependent.append(sent.text)
        else:
            independent.append(sent.text)
    
    random.shuffle(independent)
    
    reordered = independent + dependent
    return " ".join(reordered)

input_text = "AI is advancing rapidly. Researchers are exploring its ethical implications. Public awareness is growing. Because of these advances, regulations are needed."
perturbed_output = reorder_sentences(input_text)
print("Original:", input_text)
print("Perturbed:", perturbed_output)


Original: AI is advancing rapidly. Researchers are exploring its ethical implications. Public awareness is growing. Because of these advances, regulations are needed.
Perturbed: Public awareness is growing. AI is advancing rapidly. Researchers are exploring its ethical implications. Because of these advances, regulations are needed.
